In [1]:
import sys
import numpy as np
import pandas as pd
sys.path.append("../code/")

In [2]:
from dataloader import APPLIANCE_ORDER, get_train_test
from ddsc import SparseCoding, reshape_for_sc

In [3]:
num_folds = 5

In [4]:
def discriminative(dataset, num_latent, num_iterations):
    print(num_iterations)
    out = []
    for cur_fold in range(5):
        train, test = get_train_test(dataset, num_folds=num_folds, fold_num=cur_fold)
        train_sc, test_sc = reshape_for_sc(train), reshape_for_sc(test)
        train_data = np.array([train_sc[:, :, i ] for i in range(1, train.shape[1])]).swapaxes(1, 2)
        c = SparseCoding()
        c.train(train_data, num_latent=num_latent)
        pred = c.disaggregate_discriminative(train_sc[:, :, 0].swapaxes(0, 1), 
                                             test_sc[:, :, 0].swapaxes(0, 1),
                                             num_iter=num_iterations)
        pred = pred[-1, :, :, :]
        pred = pred.swapaxes(0, 2).swapaxes(1, 2)
        print(pred.shape)
        pred = pred.reshape(pred.shape[0], pred.shape[1], -1, 24)
        out.append(pred)
    return np.concatenate(out)


In [23]:
# tensor = np.load('../2015-5appliances.numpy.npy')
tensor = np.load("../2015-5appliances-subtract-true-agg.npy")
dataset = 3
from sklearn.metrics import mean_absolute_error

err_disc ={}
gt = tensor[:, 1:, :, :]
# should be varied from [1, 30]
pred = {}
for num_latent in range(5,7):
    err_disc[num_latent] = {}
    pred[num_latent] = {}
    # Should be upto 100 or so iterations.
    for num_iterations in [10, 20, 50, 80]:
        print(num_latent, num_iterations)
        pred[num_latent][num_iterations] = discriminative(dataset, num_latent, num_iterations)
        # Clamping prediction to aggregate
        pred[num_latent][num_iterations] = np.minimum(pred[num_latent][num_iterations], tensor[:, 0:1, :,:])
        err_disc[num_latent][num_iterations] = {APPLIANCE_ORDER[i+1]:mean_absolute_error(pred[num_latent][num_iterations][:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(pred[num_latent][num_iterations].shape[1])}

5 10
10
(14, 5, 2688)
(14, 5, 2688)
(14, 5, 2688)
(13, 5, 2688)
(13, 5, 2688)
5 20
20
(14, 5, 2688)
(14, 5, 2688)
(14, 5, 2688)
(13, 5, 2688)
(13, 5, 2688)
5 50
50
(14, 5, 2688)
(14, 5, 2688)
(14, 5, 2688)
(13, 5, 2688)
(13, 5, 2688)
5 80
80
(14, 5, 2688)
(14, 5, 2688)
(14, 5, 2688)
(13, 5, 2688)
(13, 5, 2688)
6 10
10
(14, 5, 2688)
(14, 5, 2688)
(14, 5, 2688)
(13, 5, 2688)
(13, 5, 2688)
6 20
20
(14, 5, 2688)
(14, 5, 2688)
(14, 5, 2688)
(13, 5, 2688)
(13, 5, 2688)
6 50
50
(14, 5, 2688)
(14, 5, 2688)
(14, 5, 2688)
(13, 5, 2688)
(13, 5, 2688)
6 80
80
(14, 5, 2688)
(14, 5, 2688)
(14, 5, 2688)
(13, 5, 2688)
(13, 5, 2688)


In [28]:
pd.Panel(err_disc).mean()

,5,6
10,185.476551,187.126524
20,183.929598,190.365883
50,187.766286,186.539268
80,192.487479,187.825474


In [29]:
err_disc[6]

,10,20,50,80
dr,107.174086,112.152993,111.398619,113.162509
dw,38.092549,48.621393,47.670876,49.599447
fridge,177.144817,110.391784,130.978013,172.687560
hvac,557.152371,573.158100,552.965780,543.500960
mw,56.068799,107.505148,89.683051,60.176896


In [ ]:
# Min occurs at #latent =1, # iterations = 3

In [10]:
res_sparse_disc = err_disc[1][3]

In [11]:
res_sparse_disc

dr         85.580721
dw         15.711825
fridge     85.830357
hvac      677.510031
mw          9.274755
Name: 3, dtype: float64